In [1]:
from dataset import AarizDataset
import torch
path = "Aariz Dataset"
dataset = AarizDataset( path ,"TRAIN")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.__version__)
print(torch.version.cuda)         # should be like "12.1" / "12.4", not None
print(torch.backends.cuda.is_built())
print(torch.cuda.is_available())


2.10.0+cu126
12.6
True
True


In [8]:
from PIL import Image
import os


folder = "Aariz Dataset/train/Cephalograms"

max_width = max_height = 0

num_folders = 0

for f in os.listdir(folder):
    num_folders += 1
    if f.lower().endswith(".png") or f.lower().endswith(".jpg") or f.lower().endswith(".jpeg") or f.lower().endswith(".bmp"):
        with Image.open(os.path.join(folder, f)) as img:
            w, h = img.size
            if max_height == 0:
                print(h,w)
            max_height = max(max_height, h)
            max_width = max(max_width, w)

print("Max width:", max_width)
print("Max height:", max_height)
print("Number of files:", num_folders)

2225 1968
Max width: 2198
Max height: 2719
Number of files: 700


In [6]:
image, landmarks, cvm = dataset[120]
print(image.size())
print(landmarks.size())

torch.Size([3, 2300, 2200])
torch.Size([29, 2])


In [4]:
import cv2
from config import ANATOMICAL_LANDMARKS

def draw_points(image,landmarks, scale=0.3,
                point_radius =5, point_color = (0.0,255), point_thickness=-1,
                 offset =6, font = cv2.FONT_HERSHEY_SIMPLEX, font_scale = 0.5, label_color=(0,0,255)):

    # ---- tensor → numpy ----
    img = image.detach().cpu().numpy()

     # handle channel-first tensors: (3, H, W) RGB -> (H,W,3)
    img = img.transpose(1, 2, 0)

           # ---- resize for display ----
    img = cv2.resize(img, (0, 0), fx=scale, fy=scale)

    points = landmarks.detach().cpu().numpy()

    landmark_symbols = [ v["symbol"] for v in ANATOMICAL_LANDMARKS.values() ]

    for i, point in enumerate(points):
        x,y = point
                # scaled landmark
        x_s = int(x * scale)
        y_s = int(y * scale)

        label = landmark_symbols[i]

        cv2.circle(img, (x_s, y_s), radius = point_radius, color = point_color, thickness = point_thickness)
        cv2.putText(img, label, (x_s+ offset, y_s- offset),
                font, font_scale, label_color)

    return img




# (H,W,3)


# ---- draw landmark ---- # color

draw_points(image, landmarks)


# ---- show ----
cv2.imshow("check", img)
cv2.waitKey(0)
cv2.destroyAllWindows()






NameError: name 'image' is not defined

In [4]:

import cv2
import numpy as np

label = "A Point"

def draw_labeled_point(img, x, y, label, point_color = (0.0,255), label_color=(0,0,0)):
    cv2.circle(img, (x, y), 5, point_color, -1)
    cv2.putText(img, label, (x+6, y-6),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, label_color, 1)

x, y = 1313, 1133

# scale factor for display only
scale = 0.3

# ---- tensor → numpy ----
img = image.detach().cpu().numpy()

# handle channel-first tensors
# (3, H, W) RGB
img = img.transpose(1, 2, 0)
# (H,W,3)

# ---- resize for display ----
img_disp = cv2.resize(img, (0, 0), fx=scale, fy=scale)

# scaled landmark
x_s = int(x * scale)
y_s = int(y * scale)

# ---- draw landmark ---- # color


draw_labeled_point(img_disp, x_s, y_s,label)

# ---- show ----
cv2.imshow("check", img_disp)
cv2.waitKey(0)
cv2.destroyAllWindows()






In [1]:
import torch
from torch.utils.data import DataLoader
from dataset import AarizDataset
path = "Aariz Dataset"
dataset = AarizDataset( path ,"TRAIN")

batch_size = 64
training_data = AarizDataset( path ,"TRAIN")


# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size,shuffle=True)

In [2]:
for (images, landmarks,_) in train_dataloader:
    print(f"Shape of Image Batch: {images.shape}")
    print(f"Shape of Landmark Batch: {landmarks.shape} {landmarks.dtype}")
    break


Shape of Image Batch: torch.Size([64, 3, 2750, 2200])
Shape of Landmark Batch: torch.Size([64, 29, 2]) torch.float32


In [ ]:
import json

junior_path = "Aariz Dataset/train/Annotations/Cephalometric Landmarks/Junior Orthodontists"
senior_path = "Aariz Dataset/train/Annotations/Cephalometric Landmarks/Senior Orthodontists"

with open(junior_path, mode="r") as file:
    junior_annotations = json.load(file)

with open(senior_path, mode="r") as file:
    senior_annotations = json.load(file)

